In [18]:
# imports:
import torch
import numpy as np
from torchvision.utils import save_image
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
import models



In [26]:
# Creating a noisy random tensor
# -----------------------------------------------------------Milestone 1-------------------------------------------------------------
def create_image():
    input = torch.rand(3,224,224)
    new_input = normalize_input(input)
    new_input = new_input.requires_grad_(True)

    #mean1,std1 = new_input.mean([1,2]), new_input.std([1,2])
    #print("Mean of the image after norm:", mean1)
    #print("Std of the image after norm:", std1)
    
    return new_input

In [27]:
#Pre-Processing for the input image
def normalize_input(input):
    mean,std = input.mean([1,2]), input.std([1,2])
    print("Mean of the image:", mean)
    print("Std of the image:", std)
    norm = transforms.Normalize(mean, std)
    new_input = norm(input)

    return new_input

In [38]:
# Create a random input for the Generator
# -----------------------------------------------------------Milestone 2-------------------------------------------------------------
def create_input():
    input_vector = torch.rand(1,8,1)
    input_vector = input_vector.requires_grad_(True)

    return input_vector

In [32]:
# Loading the Generator
# -----------------------------------------------------------Milestone 2-------------------------------------------------------------
import sys
sys.path.append('/GANLatentDiscovery')

#import GANLatentDiscovery.models.gan_load
#import GANLatentDiscovery.models.BigGAN.BigGAN
#import GANLatentDiscovery.models.BigGAN.utils

import GANLatentDiscovery.loading
generator = GANLatentDiscovery.loading.load_from_dir("/models/G_ema.pth")
#print(generator)

ModuleNotFoundError: No module named 'models.gan_load'

In [40]:
# Importing the WideResNet-50 robust ImageNet model:             
from robustbench.utils import load_model
model = load_model(model_name='Salman2020Do_R18', dataset='imagenet', threat_model='Linf')
#print(model.fc[1])

In [41]:
# Defining an Optimizer
def define_optimizer(input):
    optimizer = optim.Adam([input], lr = 0.08) 
    return optimizer                                                             # Read about Adam/SGD optimizers

In [42]:
# Plot the input image:
def visualize(input):  
    npimg = input.detach().numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

In [7]:
# Plot the input image after normalization to [0,1]:
def visualize_with_norm(input):  

    npimg = input.detach().numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

    mean,std = input.mean([1,2]), input.std([1,2])
    print("Mean of the image:", mean)
    print("Std of the image:", std)
#....................................Normalize the input to [0,1] here...................................
    new_input = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])(input)

    mean1,std1 = new_input.mean([1,2]), new_input.std([1,2])
    print("Mean of the image after norm:", mean1)
    print("Std of the image after norm:", std1)
     
    npimg = new_input.detach().numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

In [12]:
# "Training": (Not really. We optimize the input image, not the model)
# -----------------------------------------------------------Milestone 1-------------------------------------------------------------
def training(input, optimizer, class_number):
    
    for epoch in range(101): 
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = F.softmax(model(input.unsqueeze(0)),dim = 1)
        loss = 1 - output[0][class_number]     # output[0][1] = The probability that the input is an image of a goldfish. 
        loss.backward()
        optimizer.step()

        #print(loss.item())
        #print(output)
        
        if (epoch%10 == 0):
            # Show the tensor as an image and see the progress:
            print("\nEpoch #", epoch,":\n")
            visualize(input)
            print("Loss: ",loss.item())

    print('Finished Training\n\n\n')
    visualize_with_norm(input)


In [43]:
# "Training": (Not really. We optimize the input of the Image Generator, not the model itself) 
# -----------------------------------------------------------Milestone 2-------------------------------------------------------------
def training2(input, optimizer, class_number):
    
    for epoch in range(101): 
        # zero the parameter gradients
        optimizer.zero_grad()

        generated_image = generator(input)

        # forward + backward + optimize
        output = F.softmax(model(generated_image.unsqueeze(0)),dim = 1)
        loss = 1 - output[0][class_number]     # output[0][1] = The probability that the input is an image of a goldfish. 
        loss.backward()
        optimizer.step()

        
        if (epoch%10 == 0):
            # Show the tensor as an image and see the progress:
            print("\nEpoch #", epoch,":\n")
            print("\nInput: ", input)
            visualize(generated_image)
            print("Loss: ",loss.item())

    print('Finished Training\n\n\n')
    # visualize_with_norm(generated_image)
    return generated_image


In [ ]:
# We want an image of a goldfish:
# -----------------------------------------------------------Milestone 1-------------------------------------------------------------
input = create_image()
training(input, define_optimizer(input), 1)
save_image(input, "goldfish.png")

In [ ]:
# We want an image of a goldfish: 
# -----------------------------------------------------------Milestone 2-------------------------------------------------------------
input_vector = create_input()
generated_image = training2(input_vector, define_optimizer(input_vector), 1)
save_image(generated_image, "goldfish.png")

In [ ]:
# We want an image of a banana :
input = create_image()
training(input, define_optimizer(input), 954)
save_image(input, "banana.png")

In [ ]:
# We want an image of a golden retriever :
input = create_image()
training(input, define_optimizer(input), 207)
save_image(input, "golden_retriever.png")


In [ ]:
# We want an image of a tabby cat :
input = create_image()
training(input, define_optimizer(input), 281)
save_image(input, "tabby_cat.png")
